In [1]:
import sqlite3
import pandas as pd

df = pd.read_csv('online_retail_cleaned.csv')

# Create a connection to a virtual database in memory
conn = sqlite3.connect(':memory:')

# Write the dataframe to a SQL table named 'sales'
df.to_sql('sales', conn, index=False, if_exists='replace')

print("SQL Database is ready! You can now write SQL queries against the 'sales' table.")

SQL Database is ready! You can now write SQL queries against the 'sales' table.


In [3]:
query_monthly_revenue = """
SELECT 
    strftime('%Y-%m', invoicedate) AS month,
    ROUND(SUM(total_sales), 2) AS monthly_revenue
FROM sales
GROUP BY month
ORDER BY month;
"""

monthly_rev_df = pd.read_sql(query_monthly_revenue, conn)
print(monthly_rev_df)

      month  monthly_revenue
0   2009-12        683504.01
1   2010-01        555802.67
2   2010-02        504558.96
3   2010-03        696978.47
4   2010-04        591982.00
5   2010-05        597833.38
6   2010-06        636371.13
7   2010-07        589736.17
8   2010-08        602224.60
9   2010-09        829013.95
10  2010-10       1033112.01
11  2010-11       1166460.02
12  2010-12        570422.73
13  2011-01        568101.31
14  2011-02        446084.92
15  2011-03        594081.76
16  2011-04        468374.33
17  2011-05        677355.15
18  2011-06        660046.05
19  2011-07        598962.90
20  2011-08        644051.04
21  2011-09        950690.20
22  2011-10       1035642.45
23  2011-11       1156205.61
24  2011-12        517208.44


In [4]:
query_top_customers = """
SELECT 
    customer_id,
    ROUND(SUM(total_sales), 2) AS total_spent,
    COUNT(DISTINCT invoice) AS number_of_orders
FROM sales
GROUP BY customer_id
ORDER BY total_spent DESC
LIMIT 10;
"""

top_customers_df = pd.read_sql(query_top_customers, conn)
print(top_customers_df)

   customer_id  total_spent  number_of_orders
0      18102.0    580987.04               145
1      14646.0    528602.52               151
2      14156.0    313437.62               156
3      14911.0    291420.81               398
4      17450.0    244784.25                51
5      13694.0    195640.69               143
6      17511.0    172132.87                60
7      16446.0    168472.50                 2
8      16684.0    147142.77                55
9      12415.0    144458.37                28


In [6]:
from scipy import stats
import numpy as np

df['quantity_zscore'] = np.abs(stats.zscore(df['quantity']))
anomalies = df[df['quantity_zscore'] > 3]

print(f"Detected {len(anomalies)} anomalies in order quantities.")

Detected 1180 anomalies in order quantities.


In [8]:
df_normal = df[df['quantity_zscore'] <= 3].copy()

print(f"Normal dataset size: {len(df_normal)}")

Normal dataset size: 778245


In [10]:
df_normal['invoicedate'] = pd.to_datetime(df_normal['invoicedate'])
import datetime as dt
latest_date = df_normal['invoicedate'].max() + dt.timedelta(days=1)
# Proceed with the RFM grouping
rfm = df_normal.groupby('customer_id').agg({
    'invoicedate': lambda x: (latest_date - x.max()).days, # Recency
    'invoice': 'count',                                   # Frequency
    'total_sales': 'sum'                                  # Monetary
})
# Rename columns for clarity
rfm.columns = ['recency', 'frequency', 'monetary']
print(rfm.head())

             recency  frequency  monetary
customer_id                              
12346.0          529         33    372.86
12347.0            2        222   4921.53
12348.0           75         51   2019.40
12349.0           19        175   4428.69
12350.0          310         17    334.40


In [11]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 1. Scale the data
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm)

# 2. Apply K-Means (Let's start with 4 clusters: VIP, Loyal, At Risk, New)
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
rfm['cluster'] = kmeans.fit_transform(rfm_scaled).argmax(axis=1) 

print(rfm.groupby('cluster').mean())

            recency    frequency       monetary
cluster                                        
0          3.200000  6461.800000  328361.604000
2        201.682827   127.336292    2489.974455


In [12]:
cluster_map = {
    0: 'VIP / Top Spenders',
    1: 'Loyal / Regulars',
    2: 'At Risk / Hibernating',
    3: 'New / Potential'
}

rfm['segment_name'] = rfm['cluster'].map(cluster_map)

# See how many customers are in each group
print(rfm['segment_name'].value_counts())

segment_name
At Risk / Hibernating    5858
VIP / Top Spenders          5
Name: count, dtype: int64


In [13]:
rfm.to_csv('customer_segments.csv')